# 每日收盤行情

* 至 [台灣證券交易所/交易資訊/每日收盤行情](https://www.twse.com.tw/zh/page/trading/exchange/MI_INDEX.html)頁面，分類選擇「全部(不含權證、牛熊證、可展廷牛熊證)」
* 找到 csv 檔案的下載網址
* 使用 requests & pandas 套件下載及讀取資料
* 將資料經適當的處理 (轉換格式、刪除等)
* 儲存至資料庫

In [1]:
# 輸入擷取日期

date = input('請輸入擷取日期（格式:20200327）：')

In [2]:
# 使用例外判斷擷取資料
import requests

url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+date+'&type=ALLBUT0999'

try:
    data = requests.get(url)
except Exception as e:
    print('**WARRN: 擷取失敗')
    print('錯誤訊息', e)
else:
    print('成功擷取！')

In [3]:
# 觀看擷取內容

data.text[:1000]

In [4]:
# 依換行 \n 符號分割擷取資料（取得每一列資料）
# 用以判斷哪些列資料是要保留的個股每日收盤資料

lines = data.text.split('\n')
len(lines)  #取得行數

In [5]:
# 觀看某一列資料

lines[194]

In [6]:
# 判斷每一列的欄位數

print(len(lines[3].split(',"')))
print(len(lines[194].split(',"')))  #此列開始才是要保留的資料

In [7]:
# 保留欄位數 > 10 的列資料

newdata = []
for line in lines:
    if len(line.split(',"')) > 10:
        newdata.append(line)
print(len(newdata))
print(newdata[0])
print(newdata[1])

In [8]:
# 轉換 list 型別為 str

print(type(newdata))
data = '\n'.join(newdata)
print(type(data))
data[:300]

In [9]:
# 移除股票代號前的等號 

data = data.replace('=', '')
data[:300]

In [10]:
# pandas.read_csv 需要 file-like 物件，不可為 string，所以需使用 StringIO 進行轉換 
# 將 str 型號轉為 memory file (string buffer)

from io import StringIO

data_io = StringIO(data)
print(type(data_io))
#StringIO.getvalue(data_io)

In [11]:
import pandas as pd

df = pd.read_csv(data_io)
df.head(5)

In [12]:
# 刪除不需要的欄位

df.drop('漲跌(+/-)', axis=1, inplace=True)
df.drop('漲跌價差', axis=1, inplace=True)
df.drop('本益比', axis=1, inplace=True)
df.drop('Unnamed: 16', axis=1, inplace=True)
df.head(5)

In [13]:
# 移除資料裡的逗號

# def func(s):
#     #print(s)
#     return s.str.replace(',', '')

# df = df.apply(func)

df = df.apply(lambda s:s.str.replace(',',''))
df.head(5)

In [14]:
# 除了證券代號與證券名稱，其餘欄位資料型別應為數字
# 先將證券代號與證券名稱設為索引，以利後續資料型別的轉換

df.set_index(['證券代號','證券名稱'], inplace=True)
df.head(5)
df.dtypes  #觀看欄位資料型別

In [15]:
# 將欄位資料由 str 型別轉為數字型別

df = df.apply(lambda s:pd.to_numeric(s, errors='coerce'))
df.dtypes

In [16]:
# 移除資料不完整者

print('移除前:',len(df))
df.loc[df['收盤價'].isnull()]  # 找出沒有股價資料者
df = df.loc[~df['收盤價'].isnull()]  # 將沒有股價資料移除
print('移除後:', len(df))

In [17]:
# 新增日期欄位

df['date'] = pd.to_datetime(date, format='%Y%m%d', errors='ignore')
df.head(5)

In [18]:
# 重設索引及更換欄位名稱

df = df.reset_index()
df = df.rename(columns={'證券代號':'sid'})
df.set_index(['sid','date'], inplace=True)
df.head(5)

# 寫入 SQLite 資料庫

In [19]:
# 將 dataframe 資料寫入資料庫
# 如果 price table 存在，新增資料，否則，建立表格及新增資料

import sqlite3

conn = sqlite3.connect('stock1.db')

# 判斷 price table 是否已存在
# sqlite會自動維護一個系統表 sqlite_master，該表儲存了資料庫裡的各個表格等資訊
query = "select count(*) from sqlite_master where type='table' and name='price'"
exist=list(conn.execute(query))[0][0]

if exist:
    df.to_sql('price', conn, if_exists='append')
    print(date, '資料新增完畢!')
else:
    df.to_sql('price', conn, if_exists='replace')
    print(date, '資料儲存完畢!')

conn.close()

In [20]:
# 讀取資料庫

import sqlite3

conn = sqlite3.connect('stock1.db')

sqlstr = 'select * from price'
#sqlstr = 'select 證券代號,證券名稱,收盤價 from price'
#sqlstr = 'select * from price where 收盤價 > 100'
df1 = pd.read_sql(sqlstr, conn, index_col=['sid','date'])
df1

# 資料操作

In [21]:
# 股價漲幅超過 5%

df2 = df[df['收盤價'] / df['開盤價'] > 1.05]
df2